In [5]:
import pandas as pd
import xarray as xr

In [6]:
filepath = "../premise/data/metals/metals_db.csv"

In [7]:
df = pd.read_csv(filepath)

In [8]:
df

,metal,unit,year,mean,median,min,max,origin_var
0,Dysprosium,kg/MW,2015,2.512950,2.20,0.0516,11.6,Electro Motor
1,Neodymium,kg/MW,2015,10.388654,7.50,3.4000,36.5,Electro Motor
2,Dysprosium,kg/MW,2020,2.500000,2.10,1.9000,5.6,Electro Motor
3,Neodymium,kg/MW,2020,3.700000,3.60,3.4000,4.1,Electro Motor
4,Dysprosium,kg/MW,2030,1.433333,1.40,1.3000,1.6,Electro Motor
...,...,...,...,...,...,...,...,...
261,Nickel,kg/MW,2015,270.333333,256.00,255.0000,300.0,Nuclear
262,Silver,kg/MW,2015,8.150000,8.15,8.0000,8.3,Nuclear
263,Vanadium,kg/MW,2015,0.800000,0.80,0.6000,1.0,Nuclear
264,Yttrium,kg/MW,2015,0.750000,0.75,0.5000,1.0,Nuclear


In [9]:
df = df.melt(
    id_vars=["metal", "year", "origin_var"],
    value_vars=["mean", "median", "min", "max"]
)

In [10]:
df

,metal,year,origin_var,variable,value
0,Dysprosium,2015,Electro Motor,mean,2.512950
1,Neodymium,2015,Electro Motor,mean,10.388654
2,Dysprosium,2020,Electro Motor,mean,2.500000
3,Neodymium,2020,Electro Motor,mean,3.700000
4,Dysprosium,2030,Electro Motor,mean,1.433333
...,...,...,...,...,...
1059,Nickel,2015,Nuclear,max,300.000000
1060,Silver,2015,Nuclear,max,8.300000
1061,Vanadium,2015,Nuclear,max,1.000000
1062,Yttrium,2015,Nuclear,max,1.000000


In [11]:
array = df.groupby(["metal", "origin_var", "year", "variable"]).mean()["value"].to_xarray()

In [12]:
array = array.interpolate_na(dim="year", method="nearest", fill_value="extrapolate")
array = array.bfill(dim="year")
array = array.ffill(dim="year")

In [22]:
import numpy as np
np.isnan(array.sel(metal="Cobalt", origin_var="AEL", variable="mean", year=2020).values)

False

In [1]:
from premise import *
from datapackage import Package
import brightway2 as bw

In [2]:
bw.projects.set_current("new")

In [3]:
years = [
    2005,
    2010,
    2015,
    2020,
    2025,
    2030,
    2035,
    2040,
    2045,
    2050,
    2060,
    2070,
    2080,
    2090,
    2100,
]
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": y}
    for y in years
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        #external_scenarios=[Package(url) for url in urls]
)

premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.update_all()
ndb.update_buses()
ndb.update_cars()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Extracted 1 worksheets in 7.64 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available before 2015. Hence, 2015 is used as fleet year.
Done!
Extracted 1 worksheets in 3.28 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available before 2015. Hence, 2015 is used as fleet year.
Done!
Extracted 1 worksheets in 7.32 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!
Extracted 1 worksheets in 3.50 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average

ValueError: list.remove(x): x not in list

In [ ]:
ndb.write_db_to_matrices()

In [7]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/premise/data/export/scenario_report.


In [28]:
# the directory to the set of files produced by premise
iam_model = "image"
scenario = "SSP2-Base"
years = [
    2005,
    2010,
    2015,
    2020,
    2025,
    2030,
    2035,
    2040,
    2045,
    2050,
    2060,
    2070,
    2080,
    2090,
    2100,
]

for year in years:
    DIR = Path(f"./{iam_model}/{scenario}/{year}") 

    # creates dict of activities <--> indices in A matrix
    A_inds = dict()
    with open(DIR / "A_matrix_index.csv", 'r') as read_obj:
        csv_reader = reader(read_obj, delimiter=";")
        for row in csv_reader:
            A_inds[(row[0], row[1], row[2], row[3])] = row[4]
    A_inds_rev = {int(v):k for k, v in A_inds.items()}

    # creates dict of bio flow <--> indices in B matrix
    B_inds = dict()
    with open(DIR / "B_matrix_index.csv", 'r') as read_obj:
        csv_reader = reader(read_obj, delimiter=";")
        for row in csv_reader:
            B_inds[(row[0], row[1], row[2], row[3])] = row[4]
    B_inds_rev = {int(v):k for k, v in B_inds.items()}

    # create a sparse A matrix
    A_coords = np.genfromtxt(DIR / "A_matrix.csv", delimiter=";", skip_header=1)
    I = A_coords[:, 0].astype(int)
    J = A_coords[:, 1].astype(int)
    A = sparse.csr_matrix((A_coords[:,2], (J, I)))

    # create a sparse B matrix
    B_coords = np.genfromtxt(DIR / "B_matrix.csv", delimiter=";", skip_header=1)
    I = B_coords[:, 0].astype(int)
    J = B_coords[:, 1].astype(int)
    B = sparse.csr_matrix((B_coords[:,2] * -1, (I, J)), shape=(A.shape[0], len(B_inds)))


<xarray.DataArray 'value' (notnull: 25)>
array([        nan,         nan,    6.7     ,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, 2922.333333,         nan,
        423.5     ,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,  108.08    ])
Coordinates:
    origin_var  <U3 'AEL'
    variable    <U4 'mean'
    year        int64 2020
  * notnull     (notnull) object MultiIndex
  * metal       (notnull) object 'Cadmium' 'Cerium' ... 'Yttrium' 'Zirconium'

In [9]:
for f in bw.Database("biosphere3"):
    if f["categories"] == ("natural resource", "in ground"):
        print(f["name"], ";", f["categories"][0], ";", f["categories"][1], ";", f["unit"], ";", f["code"])

Astatine, in ground ; natural resource ; in ground ; kilogram ; e58cbe2f-15da-5fbe-8899-d632e3cbdfe9
Tellurium, 0.5ppm in sulfide, Te 0.2ppm, Cu and Ag, in crude ore, in ground ; natural resource ; in ground ; kilogram ; 7a81cd45-7f4c-40b3-989c-6a65f42df999
Uranium, in ground ; natural resource ; in ground ; kilogram ; 2ba5e39b-adb6-4767-a51d-90c1cf32fe98
Gadolinium, 0.15% in bastnasite, 0.015% in crude ore, in ground ; natural resource ; in ground ; kilogram ; b878ca93-d699-421e-a4b6-f694dc627062
Sylvite, 25 % in sylvinite, in ground ; natural resource ; in ground ; kilogram ; d80610f2-df83-4e2a-9dc3-f74fced6577f
Zinc, in ground ; natural resource ; in ground ; kilogram ; be73218b-18af-492e-96e6-addd309d1e32
Pd, Pd 7.3E-4%, Pt 2.5E-4%, Rh 2.0E-5%, Ni 2.3E+0%, Cu 3.2E+0% in ore, in ground ; natural resource ; in ground ; kilogram ; 535bbc83-033b-42fe-9a68-8dc9eb420385
Praseodymium, 0.42% in bastnasite, 0.042% in crude ore, in ground ; natural resource ; in ground ; kilogram ; 909bc093-